# Implement FunkSVD

In this theory notebook a function is defined that performs **FunkSVD** (based on gradient descent). To test our algorithm, we will run it on the subset of the data you worked with earlier (SVD Intro notebook.) 

This function will be (re-)used in the coming notebooks to make predictions on our MovieTweetings data.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
%matplotlib inline

In [ ]:
# read in the datasets

movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

In [ ]:

# create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
ratings_mat

`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [4]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results for iteration
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [5]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 42.820963
2 		 14.267372
3 		 3.984565
4 		 2.822973
5 		 2.708673
6 		 2.660016
7 		 2.618455
8 		 2.576282
9 		 2.531007
10 		 2.481304


In [6]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[  9.81165454   8.83627984  10.53280563  10.40676523]
 [  8.81352847   7.35161244   9.2049134    8.61895749]
 [  8.08406234   6.97649642   8.40510709   7.50655771]
 [  9.68679438   8.57068291   9.95641963   9.39195414]
 [  8.65271027   7.50316407   8.85053266   8.52210755]
 [  6.81701725   5.82634274   7.45400123   6.76150045]
 [  9.25999492   8.08094049   9.44517056   9.33144947]
 [  8.30005693   7.29414761   8.40764179   8.25437247]
 [  8.44946834   7.2422767    8.84772225   8.65026716]
 [  7.87322898   6.89836853   8.00082849   7.52091143]
 [  8.86524849   7.75479177   9.14285582   9.03545731]
 [  9.46931195   8.09699048  10.40274757   9.49846193]
 [  9.17247381   8.18862453   9.63590174   9.37165377]
 [  6.49411992   5.81184214   7.07864364   6.56191811]
 [  9.80392701   8.63334413  10.1720083    9.6851605 ]
 [  9.59369115   8.49910738  10.20538801   9.85878151]
 [  8.64547162   7.54701889   8.23159536   8.32768397]
 [  7.52947114   6.5021781    6.82631215   7.18103316]
 [  7.9067

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [7]:
user_mat, movie_mat = user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 43.342267
2 		 13.861108
3 		 3.635091
4 		 2.667407
5 		 2.569089
6 		 2.510032
7 		 2.451895
8 		 2.388284
9 		 2.317473
10 		 2.238909
11 		 2.152440
12 		 2.058261
13 		 1.956996
14 		 1.849738
15 		 1.738057
16 		 1.623909
17 		 1.509490
18 		 1.397027
19 		 1.288555
20 		 1.185725
21 		 1.089680
22 		 1.001023
23 		 0.919869
24 		 0.845954
25 		 0.778767
26 		 0.717672
27 		 0.662004
28 		 0.611124
29 		 0.564456
30 		 0.521492
31 		 0.481793
32 		 0.444980
33 		 0.410732
34 		 0.378776
35 		 0.348883
36 		 0.320865
37 		 0.294573
38 		 0.269886
39 		 0.246713
40 		 0.224987
41 		 0.204654
42 		 0.185672
43 		 0.168007
44 		 0.151626
45 		 0.136493
46 		 0.122571
47 		 0.109817
48 		 0.098180
49 		 0.087607
50 		 0.078038
51 		 0.069409
52 		 0.061655
53 		 0.054709
54 		 0.048504
55 		 0.042975
56 		 0.038058
57 		 0.033694
58 		 0.029826
59 		 0.026402
60 		 0.023374
61 		 0.020697
62 		 0.018332
63 		 0.016243
64 		

In [8]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[  9.99999549   9.99999812   9.9999988   10.00000465]
 [  9.99999652   3.99999853   8.99999908  10.00000367]
 [  7.99999477   8.99999794   9.99999871   5.0000054 ]
 [  8.99999789   7.99999925   9.99999959  10.00000226]
 [  9.9999906    4.9999964    8.99999772   9.00000963]
 [  5.99999654   3.99999884   9.99999932   6.00000361]
 [  8.99999921   8.          10.00000012   9.00000092]
 [ 10.00000639   5.00000279   9.00000193   7.99999367]
 [  7.00000313   8.00000146  10.00000101   7.99999689]
 [  9.00000366   5.00000153   9.00000106   6.99999641]
 [  9.0000041    8.00000176  10.00000116   7.99999589]
 [  9.00000352  10.00000147  10.00000096   8.99999646]
 [  9.99999531   8.9999981    9.99999877   8.00000482]
 [  5.00000026   8.00000019   5.00000013   7.99999972]
 [ 10.00000474   8.0000019   10.00000126   9.99999528]
 [  8.99999537   8.99999819   9.9999988   10.00000469]
 [  9.00000014   8.00000004   8.00000007   7.99999995]
 [ 10.00000083   8.00000038   1.00000027   9.99999918]
 [  5.0000

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [9]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,  10.,  10.],
        [ 10.,   4.,   9.,  10.],
        [  8.,   9.,  10.,   5.],
        [  9.,   8.,  10.,  10.],
        [ 10.,   5.,   9.,   9.],
        [  6.,   4.,  10.,   6.],
        [  9.,   8.,  10.,   9.],
        [ 10.,   5.,   9.,   8.],
        [  7.,   8.,  10.,   8.],
        [  9.,   5.,   9.,   7.],
        [  9.,   8.,  10.,   8.],
        [  9.,  10.,  10.,   9.],
        [ 10.,   9.,  10.,   8.],
        [  5.,   8.,   5.,   8.],
        [ 10.,   8.,  10.,  10.],
        [  9.,   9.,  10.,  10.],
        [  9.,   8.,   8.,   8.],
        [ 10.,   8.,   1.,  10.],
        [  5.,   6.,  10.,  10.],
        [  8.,   7.,  10.,   7.]])

In [10]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 43.828317
2 		 15.544888
3 		 4.433828
4 		 3.013101
5 		 2.845746
6 		 2.790099
7 		 2.755741
8 		 2.727077
9 		 2.699643
10 		 2.671737
11 		 2.642441
12 		 2.611098
13 		 2.577156
14 		 2.540122
15 		 2.499549
16 		 2.455042
17 		 2.406270
18 		 2.352984
19 		 2.295036
20 		 2.232403
21 		 2.165200
22 		 2.093689
23 		 2.018279
24 		 1.939500
25 		 1.857972
26 		 1.774352
27 		 1.689274
28 		 1.603287
29 		 1.516814
30 		 1.430119
31 		 1.343318
32 		 1.256408
33 		 1.169338
34 		 1.082079
35 		 0.994716
36 		 0.907512
37 		 0.820955
38 		 0.735764
39 		 0.652853
40 		 0.573254
41 		 0.498021
42 		 0.428112
43 		 0.364295
44 		 0.307070
45 		 0.256643
46 		 0.212931
47 		 0.175607
48 		 0.144158
49 		 0.117961
50 		 0.096343
51 		 0.078633
52 		 0.064202
53 		 0.052486
54 		 0.042991
55 		 0.035300
56 		 0.029066
57 		 0.024006
58 		 0.019887
59 		 0.016527
60 		 0.013775
61 		 0.011515
62 		 0.009652
63 		 0.008110
64 		

In [11]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 11.781501215617133:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [13]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 23.187146
2 		 10.818157
3 		 7.424157
4 		 5.730780
5 		 4.663163
6 		 3.914101
7 		 3.354990
8 		 2.920533
9 		 2.573378
10 		 2.290367
11 		 2.056257
12 		 1.860498
13 		 1.695443
14 		 1.555323
15 		 1.435654
16 		 1.332879
17 		 1.244143
18 		 1.167137
19 		 1.099980
20 		 1.041128


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [14]:
# Replace each of the comments below with the correct values
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = np.size(first_1000_users) - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [15]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?


---